In [1]:
import torch
import os
from PIL import Image
import clip
import os.path as osp
import os, sys
import torchvision.utils as vutils
sys.path.insert(0, '../')

from lib.utils import load_model_weights,mkdir_p
from models.RATLIP import NetG, CLIP_TXT_ENCODER

In [2]:
device = 'cpu' # 'cpu' # 'cuda:0'
CLIP_text = "ViT-B/32"
clip_model, preprocess = clip.load("ViT-B/32", device=device)
clip_model = clip_model.eval()

In [3]:
text_encoder = CLIP_TXT_ENCODER(clip_model).to(device)
# NetG parms format the cfg
netG = NetG(64, 100, 512, 256, 3, False, clip_model).to(device)
path = '../saved_models/pretrained/mymodel.pth'# your path
checkpoint = torch.load(path, map_location=torch.device('cpu'))
netG = load_model_weights(netG, checkpoint['model']['netG'], multi_gpus=False)

In [4]:
batch_size = 8
noise = torch.randn((batch_size, 100)).to(device)

In [5]:
captions = ['the small bird is grey with a black crown and black bill.\
a smaller bird with an all grey body, a black nape, and a longer sharp bill.\
the bird has a black eyering and a black bill that is long.\
this bird has a grey body color with a few patches of brown on its crown and coverts']

In [6]:
mkdir_p('./samples')

In [7]:
# generate from text
with torch.no_grad():
    for i in range(len(captions)):
        caption = captions[i]
        tokenized_text = clip.tokenize([caption]).to(device)
        sent_emb, word_emb = text_encoder(tokenized_text)
        sent_emb = sent_emb.repeat(batch_size,1)
        fake_imgs = netG(noise,sent_emb,eval=True).float()
        
        # name = f'{captions[i].replace(" ", "-")}'
        name ="bird"
        vutils.save_image(fake_imgs.data, './samples/%s.png'%(name), 
                          nrow=8, value_range=(-1, 1), normalize=True)


# generate from text in sigle img
# with torch.no_grad():
#     batch_images = []
#     for i in range(len(captions)):
#         caption = captions[i]
#         tokenized_text = clip.tokenize([caption]).to(device)
#         sent_emb, word_emb = text_encoder(tokenized_text)
#         sent_emb = sent_emb.repeat(batch_size, 1)
#         fake_imgs = netG(noise, sent_emb, eval=True).float()

#         for j in range(batch_size):
#             fake_img = fake_imgs[j]
#             batch_images.append(fake_img)

#     # sigle 
#     for idx, fake_img in enumerate(batch_images):
#         name = 'demo_%d' % (idx + 1)
#         vutils.save_image(fake_img, f'./samples/{name}.png', 
#                           value_range=(-1, 1), normalize=True)